In [1]:
import numpy as np
from sympy import ntt as sntt
from sympy import intt as sintt

In [2]:
f = np.array([1,4,0,0])
# f_hat = np.array([5, 249, 766, 522]) # para m = 769
# f_hat = np.array([0, 4, 2, 3]) # para m=5

N_t = 4

## Fermat numbers
# alpha_t = 2
# m_t = 5
# alpha = 4
# m = 17
alpha_t = 2
m_t = 257

In [3]:
def verify_params_ntt(N, alpha, p):
    n = np.arange(1, N)
    return (np.gcd(np.power(alpha,N), p) == 1) and np.all((np.gcd(np.power(alpha,n)-1, p) == 1))

In [4]:
verify_params_ntt(4, 2, 5)

True

In [5]:
verify_params_ntt(4, 4, 17)

True

In [6]:
verify_params_ntt(8, 2, 257)

True

In [86]:
## From https://medium.com/geekculture/euclidean-algorithm-using-python-dc7785bb674a
def extended_euclidean(a, b):
    if b == 0:
        gcd, s, t = a, 1, 0
        return (gcd, s, t)  
    else:    
        s2, t2, s1, t1 = 1, 0, 0, 1   
        while b > 0:      
            q= a // b      
            r, s, t = (a - b * q),(s2 - q * s1),( t2 - q * t1)      
            a,b,s2,t2,s1,t1=b,r,s1,t1,s,t    
        gcd,s,t=a,s2,t2    
        return (gcd,s,t)

def get_inv(x, m):
    if np.gcd(x, m)!=1: raise Exception(f'{x, m} They are NOT coprimes !!')
    
    _, x, _ = extended_euclidean(x,m)
    
    return np.mod(x, m)

# About 2-D NTT

In [8]:
# a_b2 = np.array([[0,0,1,0, 0,0,0,0,0,0,0,0,0,0,0,0],
#                  [1,0,0,0, 0,0,0,0,0,0,0,0,0,0,0,0],
#                  [0,0,0,0, 0,0,0,0,0,0,0,0,0,0,0,0],
#                  [0,0,0,0, 0,0,0,0,0,0,0,0,0,0,0,0]])

# b_b2 = np.array([[0,1,0,0, 0,0,0,0,0,0,0,0,0,0,0,0],
#                  [0,0,1,1, 0,0,0,0,0,0,0,0,0,0,0,0],
#                  [0,0,0,0, 0,0,0,0,0,0,0,0,0,0,0,0],
#                  [0,0,0,0, 0,0,0,0,0,0,0,0,0,0,0,0]])

In [9]:
def _NTT2D(x, alpha1, alpha2, M, N, F_t, k, l):
    gsum = 0
    for m in range(M):
        for n in range(N):
            gsum +=np.mod(x[m,n] * np.power(alpha1, np.mod(m*k, M)) * np.power(alpha2, np.mod(n*l, N)), F_t)
    return np.mod(gsum, F_t)

def _INTT2D(x, alpha1, alpha2, M, N, F_t, m, n):
    gsum = 0
    inv_M = get_inv(M, F_t)
    inv_N = get_inv(N, F_t)
    for k in range(M):
        for l in range(N):
            gsum +=np.mod(inv_M * inv_N * x[k,l] * np.power(alpha1, np.mod(-m*k, M)) * np.power(alpha2, np.mod(-n*l, N)), F_t)
    return np.mod(gsum, F_t)

In [10]:
def NTT2D(x):
    L_L = []
    for k in range(x.shape[0]):
        L = []
        for l in range(x.shape[1]):
            L.append(_NTT2D(x, 2, 2, x.shape[0], x.shape[1], 257, k, l))
        L_L.append(L)
    
    return np.array(L_L)

def INTT2D(hat_x):
    L_L = []
    for m in range(hat_x.shape[0]):
        L = []
        for n in range(hat_x.shape[1]):
            L.append(_INTT2D(hat_x, 2, 2, hat_x.shape[0], hat_x.shape[1], 257, m, n))
        L_L.append(L)
    
    return np.array(L_L)

In [12]:
#INTT2D(np.multiply(NTT2D(a_b2), NTT2D(a_b2)))

In [13]:
#np.multiply(NTT2D(a_b2), NTT2D(a_b2)).shape

# Proof #NTT

In [14]:
def _NTT(f, n, N, alpha, m):
    gsum=0
    for k in range(0, N):
        gsum += f[k]*np.power(alpha, n*k)
    
    return np.mod(gsum, m)

def NTT_base(f, alpha, m):
    N = len(f)
    l_ret = []
    for n in range(0, N):
        l_ret.append(_NTT(f, n, N, alpha, m))
    
    return np.array(l_ret)

In [15]:
def NTT_slow(f, alpha, p):
    f = np.asarray(f, dtype=int)
    N = f.shape[0]
    
    n = np.arange(N)
    k = n.reshape((N, 1))
    
    if not verify_params_ntt(N, alpha, p): raise Exception('The parameters alpha, p and N are not coprimes !!')
    
    M = np.power(alpha, n*k)
    
    return np.mod(np.dot(M, f), p)

In [16]:
%timeit sntt(f, 5)
%timeit NTT_base(f, 2, 5)
%timeit NTT_slow(f, 2, 5)

np.allclose(NTT_slow(f, 2, 5), NTT_base(f, 2, 5))

137 µs ± 223 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
226 µs ± 3.33 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
137 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


True

In [18]:
f_hat = NTT_slow(f, 2, 5)

# Proof # INTT

In [19]:
def _INTT(f_h, k, N, alpha, m):
    inv_N = get_inv(N, m)
    gsum=0
    for n in range(0, N):
        gsum += inv_N * f_h[n]*np.power(alpha, np.mod(-n*k, N))
    
    return np.mod(gsum, m)

def INTT_base(f_h, alpha, m):
    N = len(f_h)
    l_ret = []
    for k in range(0, N):
        l_ret.append(_INTT(f_h, k, N, alpha, m))
    
    return np.array(l_ret)

In [21]:
def INTT_slow(f, alpha, p):
    f = np.asarray(f, dtype=int)
    N = f.shape[0]
    inv_N = get_inv(N, p)
    
    n = np.arange(N)
    k = n.reshape((N, 1))
    
    if not verify_params_ntt(N, alpha, p): raise Exception('The parameters alpha, p and N are not coprimes !!')
    
    M = np.power(alpha, np.mod(-n*k, N))
    
    return np.mod(inv_N * np.dot(M, f), p)

In [22]:
%timeit sintt(f_hat, 5)
%timeit INTT_base(f_hat, 2, 5)
%timeit INTT_slow(f_hat, 2, 5)

np.allclose(INTT_slow(f_hat, 2, 5), INTT_base(f_hat, 2, 5))

149 µs ± 702 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
503 µs ± 4.14 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
195 µs ± 9.16 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


True

In [23]:
f_n = INTT_slow(f_hat, 2, 5)

In [24]:
f_n

array([1, 4, 0, 0])

## Test 2D matrix

In [49]:
a_b2 = np.array([[0,1,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,1, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0]])

b_b2 = np.array([[0,0,1,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [1,1,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0],
                 [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0]])

In [50]:
alpha_t = 2
m_t = 257
#verify_params_ntt(8, 2, 257)

In [51]:
a_b2_x = np.apply_along_axis(NTT_slow, 1, a_b2, alpha_t, m_t)

In [52]:
a_b2_xy = np.apply_along_axis(NTT_slow, 0, a_b2_x, alpha_t, m_t)

In [53]:
b_b2_x = np.apply_along_axis(NTT_slow, 1, b_b2, alpha_t, m_t)

In [54]:
b_b2_xy = np.apply_along_axis(NTT_slow, 0, b_b2_x, alpha_t, m_t)

In [55]:
c_h = np.multiply(a_b2_xy, b_b2_xy)

In [56]:
c_x = np.apply_along_axis(INTT_slow, 1, c_h, alpha_t, m_t)

In [57]:
c_xy = np.apply_along_axis(INTT_slow, 0, c_x, alpha_t, m_t)

In [58]:
c_xy

array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

# Proof #1
$$\hat{f}^{\prime}_{n} = \sum^{N-1}_{k=1}{(kf_{k}) \alpha^{n(k-1)}} \mod{m}, \quad n=0, \ldots, N-1$$

In [ ]:
for n in range(N_t):
    sum_o = 0
    for k in range(1,N_t):
        sum_o += np.mod(k * f[k] * np.power(alpha_t, n*(k-1)), m_t)
    print(f'n={n} f_hat_p={sum_o}')

# Proof #2

$$\hat{f}^{\prime}_{n} = \sum^{N-1}_{k=1}{k \left(N^{-1} \sum_{l=0}^{N-1} \hat{f}_{l} \alpha^{-l k}\right) \alpha^{n(k-1)}} \mod{m}, \quad n=0, \ldots, N-1$$

In [ ]:
N_inv_t = get_inv(N_t, m_t)

for n in range(N_t):    
    sum_k = 0
    for k in range(1,N_t):
        
        sum_l = 0
        for l in range(N_t):
            sum_l += f_hat[l]*np.power(alpha_t, -l*k%N_t ) ## change 1
        sum_l *= N_inv_t ## change 2
        
        sum_k += k*sum_l*np.power(alpha_t, n*(k-1))
    sum_k = np.mod(sum_k, m_t)

    print(f'n={n} f_hat_p={sum_k}')

# Proof #3

$$\hat{f}^{\prime}_{n} = N^{-1}\sum_{l=0}^{N-1} \hat{f}_{l}\sum^{N-1}_{k=1} k \alpha^{k(n-l)}\alpha^{-n} \mod{m}, \quad n=0, \ldots, N-1$$

In [ ]:
for n in range(N_t):    
    
    sum_l = 0
    for l in range(N_t):
        
        sum_k = 0
        for k in range(1,N_t):
            sum_k += k*np.power(alpha_t, (k*(n-l))%N_t ) ## change 1

        sum_l += f_hat[l]*sum_k
    
    sum_l = N_inv_t*sum_l*np.power(alpha_t, -n%N_t) ## change 2 and 3
    sum_l = np.mod(sum_l, m_t)

    print(f'n={n} f_hat_p={sum_l}')

# Proof #4

$$\hat{f}^{\prime}_{n} =N^{-1}\sum_{l=0}^{N-1} \hat{f}_{l}\alpha^{-n}  \sum^{N-1}_{k=1} k \alpha^{k(n-l)}  \mod{m}, \quad n=0, \ldots, N-1$$

In [ ]:
for n in range(N_t):    
    
    sum_l = 0
    for l in range(N_t):
        
        sum_k = 0
        for k in range(1,N_t):
            sum_k += k*np.power(alpha_t, (k*(n-l))%N_t ) ## change 1
        
        # print(np.mod(sum_k,m))

        sum_l += f_hat[l]*sum_k
    
    sum_l = N_inv_t*np.power(alpha_t, -n%N_t)*sum_l ##change 2 and 3
    
    sum_l = np.mod(sum_l, m_t)

    print(f'n={n} f_hat_p={sum_l}')

# Proof #5

\begin{equation*}
\sum^{N-1}_{k=1} k \alpha^{k(n-l)} = \begin{cases}
  2^{-1}\left(N-1\right)N  &\text{ if } l=n  \\
  \frac{N}{\left(\alpha^{n-l} -1\right)} &\text{ if } l\neq n,
\end{cases}
\end{equation*}

In [94]:
inv_two = get_inv(2, m_t)

In [95]:
## Left 
for n in range(N_t):
    for l in range(N_t):
        sum_k=0
        for k in range(1,N_t):
            sum_k += k* np.power(alpha_t, (k*(n-l))%N_t )

        print(f'l={l},k={n}, sum= {np.mod(sum_k,m_t)}')
    print('')

l=0,k=0, sum= 1
l=1,k=0, sum= 2
l=2,k=0, sum= 3
l=3,k=0, sum= 4

l=0,k=1, sum= 4
l=1,k=1, sum= 1
l=2,k=1, sum= 2
l=3,k=1, sum= 3

l=0,k=2, sum= 3
l=1,k=2, sum= 4
l=2,k=2, sum= 1
l=3,k=2, sum= 2

l=0,k=3, sum= 2
l=1,k=3, sum= 3
l=2,k=3, sum= 4
l=3,k=3, sum= 1



In [96]:
## Right
for n in range(N_t):
    for l in range(N_t):
        if l==n:
            sum_k= inv_two*(N_t-1)*N_t
        else:
            pow_a = np.mod((n-l), N_t) ## only positive exponents
            a = np.power(alpha_t, pow_a)
            a_inv = get_inv(a-1, m_t)
            
            sum_k = N_t * a_inv


        print(f'l={l},l={n}, sum= {np.mod(sum_k,m_t)}')
    print('')

l=0,l=0, sum= 1
l=1,l=0, sum= 2
l=2,l=0, sum= 3
l=3,l=0, sum= 4

l=0,l=1, sum= 4
l=1,l=1, sum= 1
l=2,l=1, sum= 2
l=3,l=1, sum= 3

l=0,l=2, sum= 3
l=1,l=2, sum= 4
l=2,l=2, sum= 1
l=3,l=2, sum= 2

l=0,l=3, sum= 2
l=1,l=3, sum= 3
l=2,l=3, sum= 4
l=3,l=3, sum= 1



Building the T_prime matrix

In [103]:
l_l = []
for n in range(N_t):
    n_l = []
    for l in range(N_t):
        print(n,l)
        if l==n:
            t_l_n= inv_two*(N_t-1)*np.power(alpha_t,-n%N_t)
        else:
            pow_a = np.mod((n-l), N_t) ## only positive exponents
            a = np.power(alpha_t, pow_a)
            a_inv = get_inv(a-1, m_t)
            
            t_l_n = np.power(alpha_t, -n%N_t)*a_inv

        n_l.append(t_l_n)
        # print(f'l={l},l={n}, sum= {t_l_n}')
    l_l.append(n_l)

0 0
0 1
0 2
0 3
1 0
1 1
1 2
1 3
2 0
2 1
2 2
2 3
3 0
3 1
3 2
3 3


In [98]:
T_prime = np.mod(np.array(l_l),m_t)

# f_h = np.array(f_h)

np.mod(np.dot(T_prime, f_hat),m_t)

array([4, 4, 4, 4])

In [99]:
T_prime

array([[4, 3, 2, 1],
       [3, 2, 4, 1],
       [3, 4, 1, 2],
       [1, 4, 2, 3]])

# Prime Matrix

In [113]:
N_t = 4

n = np.arange(N_t)
#l = np.arange(N_t)
l = n.reshape((N_t, 1))

alpha_t = 2
m_t = 5

vget_inv = np.vectorize(get_inv)

In [101]:
l_l = []
for n in range(N_t):
    n_l = []
    for l in range(N_t):
        print(n,l)
        if l==n:
            t_l_n= inv_two*(N_t-1)*np.power(alpha_t,-n%N_t)
        else:
            pow_a = np.mod((n-l), N_t) ## only positive exponents
            a = np.power(alpha_t, pow_a)
            a_inv = get_inv(a-1, m_t)
            
            t_l_n = np.power(alpha_t, -n%N_t)*a_inv

        n_l.append(t_l_n)
        # print(f'l={l},l={n}, sum= {t_l_n}')
    l_l.append(n_l)

array([[0],
       [1],
       [2],
       [3]])

In [ ]:
vget_inv(np.power(alpha_t, np.mod((n-l), N_t))-1, m_t)

In [117]:
n

array([0, 1, 2, 3])

In [ ]:
def f(r,x):
    return np.where(x<0.5,2*r*x, 2*r*(1-x))

In [121]:
# alpha_t = 2
# m_t = 5
def ff(n,l):
    return np.piecewise([n,n],

                 [l==n,
                 l!=n],

                 [vget_inv(2, m_t)*(N_t-1)*np.power(alpha_t,np.mod(-n, N_t)),
                  1*np.power(alpha_t,np.mod(-n, N_t))]

                )

In [122]:
ff(n,n)

IndexError: boolean index did not match indexed array along dimension 0; dimension is 2 but corresponding boolean dimension is 4

In [ ]:
def Tprime(n,l,N,inv_two,p,alpha):
    return np.where(l==n,
                    inv_two*(N-1)*np.power(alpha,np.mod(-n, N)),
                    get_inv(np.power(alpha, np.mod((n-l), N))-1, p)*np.power(alpha,np.mod(-n, N)))

In [ ]:
def PrimeNTT(f_hat, alpha, p):
    f = np.asarray(f_hat, dtype=int)
    N = f.shape[0]
    
    inv_two = get_inv(2, m_t)
    
    n = np.arange(N)
    k = n.reshape((N, 1))
    
    T_prime = Tprime(n,k,N,inv_two,p,alpha)
    
    return np.mod(np.dot(T_prime, f_hat), p)

In [ ]:
#PrimeNTT(f_hat, 2, 5)